# Проект NLP

**Заказчик:** Интернет-магазин  

**Описание задачи:** запуск нового сервиса: пользователи могут редактировать и дополнять описания товаров, предлагают свои правки и комментируют изменения других. Магазину нужна модель оценки токсичности комментариев для отправки на пре-модерацию.  

**Задача:** Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

**Порог решения:** модель с *F1* не меньше 0.75.  


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyClassifier
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import transformers
import warnings
import seaborn as sns
from tqdm import tqdm
from time import time

## Подготовка

In [2]:
df = pd.read_csv('/datasets/toxic_comments.csv', index_col=False)

In [3]:
df.shape

(159292, 3)

Датасет состоит из 159к комментариев.

In [4]:
df.head(5)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [5]:
df['toxic'].value_counts().reset_index()

,index,toxic
0,0,143106
1,1,16186


16к комментариев помечены как токсичные. Присутствует дисбаланс классов

**Дубликаты**

In [6]:
mydups = df.duplicated().sum()
print('Количество полных дубликтов в датасете', mydups)

Количество полных дубликтов в датасете 0


Полные дубликаты в датасете не выявлены.

**Пропуски**

In [7]:
pd.DataFrame(round(df.isna().mean()*100,1)).style.background_gradient('bwr')

,0
Unnamed: 0,0.000000
text,0.000000
toxic,0.000000


Пропуски отсутствуют.

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


Для удобства приведём текст к lower_case

In [9]:
df['text'] = df['text'].str.lower()

Преобразуем текст в леммы.

In [21]:
def text_pre(text):
    tokenized = nltk.word_tokenize(text)
    joined = ' '.join(tokenized)
    text_o = re.sub(r"[^a-z0-9!@#\$%\^\&\*_\-,\.' ]", ' ', joined)
    final = " ".join(text_o.split())
    return  final

Добавим новый столбец с леммами в датасет.

In [11]:
import spacy

In [16]:
nlp = spacy.load('en_core_web_sm')
def lemma_clear(text): 
      
    lemm = nlp(text) 
    lemm = " ".join([token.lemma_ for token in lemm])  
    return " ".join(lemm.split())


In [19]:
tqdm.pandas() 
df['token_text'] = df['text'].progress_apply(lemma_clear)

100%|██████████| 159292/159292 [48:24<00:00, 54.84it/s]  


In [22]:
tqdm.pandas() 
df['token_text'] = df['token_text'].progress_apply(text_pre)

100%|██████████| 159292/159292 [01:34<00:00, 1679.83it/s]


Рассмотрим доработанный датасет.

In [23]:
df.head()

,Unnamed: 0,text,toxic,token_text
0,0,explanation\nwhy the edits made under my usern...,0,explanation why the edit make under my usernam...
1,1,d'aww! he matches this background colour i'm s...,0,d'aww ! he match this background colour be see...
2,2,"hey man, i'm really not trying to edit war. it...",0,"hey man , be really not try to edit war . it b..."
3,3,"""\nmore\ni can't make any real suggestions on ...",0,more can not make any real suggestion on impro...
4,4,"you, sir, are my hero. any chance you remember...",0,"you , sir , be my hero . any chance you rememb..."


В качестве фич укажем леммы, в качестве таргетов изначальную разметку.

In [24]:
features = df['token_text']
target = df['toxic']

Разделим датасет на обучающую и тестовую выборку.

In [25]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=89)

Загрузим токсичные слова на английском.

In [26]:
stopwords = nltk.corpus.stopwords.words('english')

In [27]:
count_tf = TfidfVectorizer(stop_words=stopwords)
tf_idf_new = count_tf.fit_transform(features_train)

**Ансамбль Логистической регрессии, Случайного леса и SGD моделей**

In [28]:
%%time
X_train = tf_idf_new
y_train = target_train

X_test = count_tf.transform(features_test)


log_clf = LogisticRegression(solver="liblinear", random_state=89,
                            class_weight='balanced')
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=89, 
                            class_weight='balanced')
sgb_clf = SGDClassifier(max_iter = 1000, random_state=89, 
                        loss='modified_huber')

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('sgb', sgb_clf)],
    voting='soft')

voting_clf.fit(X_train, y_train)
predict_new = voting_clf.predict(X_test)

CPU times: user 11min 32s, sys: 10.3 s, total: 11min 42s
Wall time: 11min 43s


In [30]:
f_score = f1_score(predict_new, target_test)
print(f'F1_score на тестовой выборке без DownSampling {f_score}')

F1_score на тестовой выборке без DownSampling 0.77763730861076


В рамках обучения и проверке на тесте ансамбль показал значение F1 выше порогового, что удовлетворяет условиям изначальной задачи. Применение дополнительных техник не потребовалось.

**Линейная регрессия с GS**

In [31]:
%%time
model = LogisticRegression(random_state=12345, solver='liblinear',class_weight='balanced')
parameters = {'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}

grid_clf = GridSearchCV(model, parameters, cv=3, scoring='f1')
grid_clf.fit(X_train, y_train)

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

CPU times: user 2min, sys: 2min 56s, total: 4min 57s
Wall time: 4min 57s


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=3,
             estimator=LogisticRegression(class_weight='balanced',
                                          random_state=12345,
                                          solver='liblinear'),
             param_grid={'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']},
             scoring='f1')

In [32]:
print(grid_clf.best_estimator_)
print(grid_clf.best_params_)
print(grid_clf.best_score_)

LogisticRegression(class_weight='balanced', random_state=12345)
{'solver': 'lbfgs'}
0.750709011652308


**Ансамбль Логистической регрессии, Случайного леса и SGD моделей c DownSampling**

Попробуем применить технику DownSampling для борьбы с дисбалансом классов.

In [33]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_sample = features_zeros.sample(frac=0.1, random_state=89)
    target_sample = target_zeros.sample(frac=0.1, random_state=89)
    
    features_downsampled = pd.concat([features_sample] + [features_ones])
    target_downsampled = pd.concat([target_sample] + [target_ones])
    
    features_downsampled = shuffle(features_downsampled, random_state=89)
    target_downsampled = shuffle(target_downsampled, random_state=89)
    
    return features_downsampled, target_downsampled

In [34]:
features_downsampled, target_downsampled = downsample(features_train, target_train, 0.1)

print(features_downsampled.shape)
print(target_downsampled.shape)

(22955,)
(22955,)


In [35]:
target_downsampled.value_counts().reset_index()

,index,toxic
0,1,12231
1,0,10724


В результате применения техники получили выборку меньшего размера без дисбаланса. Обучим модель.

In [36]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf = count_tf_idf.fit_transform(features_downsampled)

print(f"Размер матрицы обучения: {tf_idf.shape}")

Размер матрицы обучения: (22955, 45674)


In [37]:
X_train_ans = tf_idf
y_train_ans = target_downsampled

In [39]:
%%time
log_clf = LogisticRegression(solver="liblinear", random_state=89)
rnd_clf = RandomForestClassifier(n_estimators=10, random_state=89)
sgb_clf = SGDClassifier(max_iter = 1000, random_state=89, loss='modified_huber')

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('sgb', sgb_clf)],
    voting='soft')

X_test_ans = count_tf_idf.transform(features_test)

voting_clf.fit(X_train_ans, y_train_ans)
predict = voting_clf.predict(X_test_ans)

CPU times: user 8.13 s, sys: 5.88 s, total: 14 s
Wall time: 14 s


In [40]:
f_score_down = f1_score(predict, target_test)
print(f'F1_score на тестовой выборке c DownSampling {f_score_down}')

F1_score на тестовой выборке c DownSampling 0.655136367955906


Модель продемонстрировала на порядок лучшую скорость обучения, но в результате F1_score оказался значительно ниже порогового значения.

**Проверка модели на адекватность**

Применим Dummy модель, которая прогнозирует всем комментария токсичное значение.

In [41]:
dummy = DummyClassifier(random_state=89, strategy='constant', constant=1)

In [42]:
dummy.fit(features_train, target_train)
dummy_pred = dummy.predict(features_test)

In [43]:
f1_dummy = f1_score(target_test, dummy_pred)

print(f"F1_score константной модели {f1_dummy}")

F1_score константной модели 0.18068436200831467


Константная Dummy модель показала низкое значение F1, что в целом соотвествует изначальному балансу классов.

## Выводы

In [44]:
print(f"F1_score константной модели {f1_dummy}")
print(f'F1_score на тестовой выборке без DownSampling {f_score}')


F1_score константной модели 0.18068436200831467
F1_score на тестовой выборке без DownSampling 0.77763730861076


Пороговое значение удалось покорить на ансамбле моделей Логистической регрессии, случайного леса и SGD без применения дополнительных техник.
Техника DownSampling помогла значительно снизить скорость обучения, но это отразилось на качестве предсказаний.

**Рекомендация заказчику:** применение ансамбля моделей Логистической регрессии, случайного леса и SGD.